## Resting State Transfer Entropy Differentiates Responders and Non-Responders to Oral Ketamine Treatment for Suicidality

Authors: Mitchell, J. S., Anijärv, T-E., Can, Adem., Hermens, D.F., & Lagopoulos, J.

Code created by Jules Mitchell March 2024.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

In [22]:
# Import packages
import pandas as pd
import os 
import pickle
import numpy as np
import time

# Import classes
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from idtxl.visualise_graph import plot_network, plot_selected_vars, plot_network_comparison
from idtxl.stats import network_fdr
from idtxl import idtxl_io as io

## Choose estimator to import
# import idtxl.estimators_python # Non-linear continous data Kraskov estimator 
import idtxl.estimators_jidt

# Set directories
os.chdir('C:/Users/j_m289/Pictures/GitHub/Studies/OKTOS/RS_TransferEntropy/data')
root_dir = os.getcwd()
output_dir = '/derivatives'

# Define responders
responders = ["sub-01"]

# Initialise analysis object and define settings
network_analysis = MultivariateTE()
settings = {'cmi_estimator': 'JidtGaussianCMI', # Note, gaussian is faster but finds spurious connections in a non-linear system OpenCLKraskovCMI/JidtGaussianCMI/JidtKraskovCMI
            #'local_values': True,
            'max_lag_sources': 5,
            'min_lag_sources': 1,
            'n_perm_max_stat': 500,
            'alpha_max_stat': 0.05,
            'n_perm_min_stat': 500,
            'alpha_min_stat': 0.05,
            'permute_in_time': True,
            'perm_type': 'random',
            'n_perm_omnibus': 500,
            'alpha_omnibus': 0.05,
            'n_perm_max_seq': 500,
            'alpha_max_seq': 0.05
            #'fdr_correction': True, 
            #'alpha_fdr': 0.05,
            #'correct_by_target': True
            #'tau_sources': 2, # Subset of past samples to include (every ith sample)
            #'tau_target': 2, # Subset of past samples to include (every ith sample)
            # 'add_conditionals': 'faes' # adds the current value of all sources to the conditioning set (minimises volume conduction effects)
        }

In [ ]:
# Define functions
def load_bids(bids_folder, classifier): # change to load_bids
    # Initialize an empty list to store graphs
    bids_files = []

    # Loop through subject folders
    for subject_folder in os.listdir(bids_folder):
        subject_path = os.path.join(bids_folder, subject_folder)
        
        # Check if the subject is a responder or non-responder
        if subject_folder in classifier: # change this to a key based system
            response_category = "responder"
        else:
            response_category = "non_responder"

        # Loop through session folders
        for session_folder in os.listdir(subject_path):
            session_path = os.path.join(subject_path, session_folder)

            # Assuming graph files are stored as text files (modify as needed)
            files = [f for f in os.listdir(session_path) if f.endswith(".csv")] # Adjust file type as required

            # Load each graph file into a directed graph
            for file in files:
                file_path = os.path.join(session_path, file)

                # Load graph from file (modify based on your data format)
                df = pd.read_csv(file_path, header=None)

                # Drop channels from dataframe and select portion of resting state data
                df = df.drop(columns = 0)
                #df = df.iloc[:, 0:100] # Here is where I want the 'cut' parameter

                # Save dataframe as IDTxl data object and specify dimensions
                data = Data(df, dim_order='ps')

                if 'EC' in file_path:
                    task = 'EC'
                else:
                    task = 'EO'

                # Add the graph information to the list
                bids_files.append({
                    "subject": subject_folder,
                    "response_category": response_category,
                    "session": session_folder,
                    "task": task,
                    "data": data
                })

    return bids_files 

def combine_targets(folder, threshold = 2): # change to load_bids

    # Loop through subject folders
    for subject_folder in os.listdir(folder):
        subject_path = os.path.join(folder, subject_folder)

        # Loop through session folders
        for session_folder in os.listdir(subject_path):
            session_path = os.path.join(subject_path, session_folder)

            # Assuming graph files are stored as text files (modify as needed)
            files = [f for f in os.listdir(session_path) if f.endswith(".p")] # Adjust file type as required

            # Load results using pickle
            res_list = []

            for file in files:
                file_path = os.path.join(session_path, file)
                res_list.append(pickle.load(open(file_path, 'rb')))

            res = network_fdr({'alpha_fdr': 0.05, 'fdr_constant': threshold}, *res_list)

            # Construct the output file path
            output_dir = os.path.join('derivatives', subject_folder, session_folder)
            os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
            file_path = os.path.join(output_dir, f'{subject_folder}_{session_folder}_EO_network.p')
            
            with open(file_path, 'wb') as f:
                pickle.dump(res, f)

            print('Successfully Combined Targets for {}/{}'.format(str(subject_folder), str(session_folder)))

    return res

### Import Data

In [ ]:
# Generate mock dataset if required
data = Data ()
data.generate_mute_data(n_samples=1000, n_replications=5)

In [ ]:
# Load participant files from BIDS structure
bids_files = load_bids(root_dir, responders)

### Run analysis

In [ ]:
# Network analysis
for x in range(len(bids_files)):
    temp_data = bids_files[x]['data']
    # mTE analysis
    results = network_analysis.analyse_network(settings=settings, data=temp_data)

    # Get the subject and session information
    subject = bids_files[x]['subject']
    session = bids_files[x]['session']

    # Construct the file path
    output_dir = os.path.join('derivatives', subject, session)
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    file_path = os.path.join(output_dir, f'{subject}_{session}_EO_network.p')

    # File save
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)

In [ ]:
# Single target analysis
targets = range(1)

# Run mTE using IDTxl - current issue: Full file path not specified properly
for x in range(len(bids_files)):
    for t in targets:
        start_time = time.time()  # Record the start time

        temp_data = bids_files[x]['data']
        results = network_analysis.analyse_single_target(settings=settings, data=temp_data, target=t)

        # Get the subject and session information
        subject = bids_files[x]['subject']
        session = bids_files[x]['session']

        # Construct the file path
        output_dir = os.path.join('derivatives', subject, session)
        os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
        file_path = os.path.join(output_dir, f'{subject}_{session}_{t}_EO_target.p')

        # File save
        with open(file_path, 'wb') as f:
            pickle.dump(results, f)

        end_time = time.time()  # Record the end time
        elapsed_time = end_time - start_time  # Calculate the elapsed time
        print(f"Time taken for {subject}, {session}, target {t}: {elapsed_time:.2f} seconds")

## Combine Targets

In [ ]:
folder = 'C:/Users/j_m289/Pictures/GitHub/Studies/OKTOS/RS_TransferEntropy/data/derivatives/'
combine_targets(folder, threshold=1)

### Output Routines

In [ ]:
# BrainNet Viewer - For group comparisons of connectivity with glass brains
outfile = 'brain_net'
n_nodes = results.data_properties.n_nodes
mni_coord = np.random.randint(10, size=(n_nodes, 3))
node_color = np.random.randint(10, size=n_nodes)
node_size = np.random.randint(10, size=n_nodes)
labels = list
adj_matrix = results.get_adjacency_matrix(
    weights='max_te_lag', fdr=False,)
io.export_brain_net_viewer(adjacency_matrix=adj_matrix,
                            mni_coord=mni_coord,
                            file_name=outfile,
                            labels=labels,
                            node_color=node_color,
                            node_size=node_size)

### PBS Scripting

In [ ]:
# Generate bash script 
network_size = 32

# Define PBS script
bash_lines = '\n'.join([
    '#! /bin/bash',
    # set project name
    '#PBS -P OKTOS_TE',
    # set job name
    '#PBS -N NetorkAnalysis',
    # choose number of cores and memory
    '#PBS -l select=1:ncpus=1:mem=1GB', # select is a multiplier for ncpus and mem
    # set walltime hh:mm:ss
    '#PBS -l walltime=16:00:00',
    # set job array numbers to match network size
    '#PBS -J 0-{}'.format(network_size),
    # load Python
    'module load python/3.7.3',
    # if necessary, activate local environment where IDTxl is installed
    'source /ProjectName/idtxl_env/bin/activate',
    # run analysis on single target
    'python analyse_single_target.py $PBS_ARRAY_INDEX'
    # output directory?
    #'PBS -o myscript.out'
    ])

# Generate and save PBS script file
bash_script_name = 'parallel_analysis_using_PBS_example.pbs'
with open(bash_script_name, 'w', newline='\n') as bash_file:
    bash_file.writelines(bash_lines)